# RPC Design

## 1. RPC Protocol

### 1.1 Request

#### 1.1.1 Main Protocol

All requests about DHT hava the same resource format like the
following pattern:

```english
                                    1  1  1  1  1  1
      0  1  2  3  4  5  6  7  8  9  0  1  2  3  4  5
    +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
    |           J           |           G           |
    +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
    |                      CID                      |
    +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
    |          TYPE         |       PARAM_NUM       |
    +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
    |                     LENGTH                    |
    +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
    |                    SRC_ADDR                   |
    |                                               |
    +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
    |                                               |
    /                    FUNC_NAME                  /
    |                                               |
    +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
    |                   PARAM_PART                  |
```

JG: The magic charactors "JG"(0x4A 0x47).

CID: Call ID, unique.

TYPE: CALL(0), REGISTER(1), HELLO(2), NS(3), other TYPEs not used
    in Request.

PARAM_NUM: only non-zero when TYPE equal CALL(0).

LENGTH: len(SRC_ADDR & FUNC_NAME), max=65535 BYTES

SRC_ADDR: 32 bits.
    e.g. 192.168.1.1 => 1100_0000 1010_1000 0000_0001 0000_0001.

FUNC_NAME: ASCII string.


#### 1.1.2 Param Protocol

All requests whose TYPE equal CALL(0) hava PARAM_PART fields.

```english
                                    1  1  1  1  1  1
      0  1  2  3  4  5  6  7  8  9  0  1  2  3  4  5
    +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
    |          TYPE         |         LENGTH        |
    +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
    |                                               |
    /                     VALUE                     /
    |                                               |
    +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
```

TYPE: INT8(0), UINT8(1), INT16(2), UINT16(3), 
      INT32(4), UINT32(5), INT64(6), UINT64(7), 
      BOOL(8), FLOAT32(9), FLOAT64(10),
      BYTE(11), STRING(12), 13-255 reserverd.
      (only support the above TYPEs)

LENGTH: max=255 BYTES

VALUE: INT32 => 32; INT64 => 64;
        POINTER => 32; STRING/BITS => variable(0, 255)

### 1.2 Response

All responses have the same format like below:

```english
                                    1  1  1  1  1  1
      0  1  2  3  4  5  6  7  8  9  0  1  2  3  4  5
    +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
    |           J           |           G           |
    +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
    |                      CID                      |
    +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
    |          TYPE         |       RETURN_NUM      |
    +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
    |                   RETURN_PART                 |
```

JG: The magic charactors "JG"(0x4A 0x47).

CID: Call ID, unique.

TYPE: HELLO(2), NS(3),
      PARAMERROR(251), UNEXIST(252), AUTHFAIL(253), OK(254),
      UNKNOWN(255), 0-3 is request(2,3 both) reserverd.

RETURN_NUM: Num of Return Value.

All T-V Return-Parts own the following format:

```english
                                    1  1  1  1  1  1
      0  1  2  3  4  5  6  7  8  9  0  1  2  3  4  5
    +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
    |          TYPE         |         LENGTH        |
    +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
    |                                               |
    /                     VALUE                     /
    |                                               |
    +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
```

## 2. Chord DHT

### 2.1 Node struct for Chord

LdBlsServer maintains a ring which is comprised with several Nodes.

HID(Hash ID) is generated by SHA-1 algothrim, abstract of 160 bits.
addr uses net.IP.
port varies from 1 to 65535(2^16-1).

chordRings is a map, which its key is funcName(string) and its value
is a chordRing.
LdBlsServer do not check validation of parameters. It just work as a
load balancer, choosing a suitable node server to execute procedure.
chordRing is a data structure.

```go
var chordRings map[string][]chordNode = make(
        map[string][]chordNode, ringSize
    )

type chordNode struct {
    HID       []byte
    addr    net.IP
    port    uint16
}
```

But in my jgRPC.go, the solution using Chord to store HT is abondoned.